# Data Preprocessing

## Importing Libraries

In [33]:
#importing Libraries
import pandas as pd
import os
import sys

#import local libraries
#Adding scripts path
sys.path.append(os.path.abspath(os.path.join('..')))
#importing dvc_data_loader script
# from scripts.dvc_data_loader import *
from scripts.data_information import DataInfo
from scripts.data_loader import load_df_from_csv
from scripts.data_manipulation import DataManipulator
from scripts.data_cleaner import DataCleaner
from scripts.utlity_functions import convert_to_month_name
from scripts.grapher import *
from sklearn import preprocessing

In [34]:
pd.set_option('max_column', None)


## Loading Data

In [59]:
combined_df = load_df_from_csv('../data/train.csv')
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 22 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Unnamed: 0                 1017209 non-null  uint32 
 1   Store                      1017209 non-null  uint16 
 2   DayOfWeek                  1017209 non-null  uint8  
 3   Date                       1017209 non-null  object 
 4   Year                       1017209 non-null  uint16 
 5   Month                      1017209 non-null  uint8  
 6   Day                        1017209 non-null  uint8  
 7   Sales                      1017209 non-null  uint16 
 8   Customers                  1017209 non-null  uint16 
 9   Open                       1017209 non-null  uint8  
 10  Promo                      1017209 non-null  uint8  
 11  StateHoliday               1017209 non-null  object 
 12  SchoolHoliday              1017209 non-null  uint8  
 13  StoreType   

In [39]:
combined_df.head(5)

,Unnamed: 0,Store,DayOfWeek,Date,Year,Month,Day,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,0,1,5,2015-07-31,2015,7,31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1,1,4,2015-07-30,2015,7,30,5020,546,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,2,1,3,2015-07-29,2015,7,29,4782,523,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,3,1,2,2015-07-28,2015,7,28,5011,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,4,1,1,2015-07-27,2015,7,27,6102,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [60]:
combined_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [4]:
# Saving Holiday dates for later prediciton
holiday_days = combined_df[['Month', 'Day', 'StateHoliday']]
holiday_days = holiday_days[holiday_days['StateHoliday'] != 0]
holiday_days = holiday_days[holiday_days['StateHoliday'] != '0']
holiday_days = holiday_days.drop_duplicates()
holiday_days.drop('StateHoliday', axis=1, inplace=True)
holiday_days.to_csv('../models/holiday_reference.csv', index=False)

## Handling None and Empty Values

In [61]:
info = DataInfo(combined_df)
info.get_column_based_missing_percentage()

,total_missing_values,missing_percentage
Store,0,0.0 %
DayOfWeek,0,0.0 %
Date,0,0.0 %
Year,0,0.0 %
Month,0,0.0 %
Day,0,0.0 %
Sales,0,0.0 %
Customers,0,0.0 %
Open,0,0.0 %
Promo,0,0.0 %


In [62]:
manipulator = DataManipulator(combined_df)
# Fill missing numeric values
manipulator.fill_columns_with_max(info.get_numeric_columns())
# Fill non-numeric values (categorical values)
manipulator.fill_columns_with_most_frequent(info.get_object_columns())


In [63]:
info.get_columns_with_missing_values()

[]

## Adding Additional Extracted Data

### WeekDays

In [64]:
manipulator.add_week_day('DayOfWeek')

Data Manipulatior:INFO->Successfully Added WeekDay Column to the DataFrame


### WeekEnds


> Doesnt Add additional information

In [65]:
# combined_df.add_week_ends(combined_df, 'DayOfWeek')


### Number of days to Holidays


In [66]:
manipulator.add_number_of_days_to_holiday('StateHoliday')


Data Manipulatior:INFO->Successfully Added DaysToHoliday Column


### Number of days after Holiday


In [67]:
manipulator.add_number_of_days_after_holiday('StateHoliday')


Data Manipulatior:INFO->Successfully Added DaysAfterHoliday Column


### Beginning of month, mid month and ending of month


In [68]:
manipulator.add_month_timing('Day')


Data Manipulatior:INFO->Successfully Added MonthTiming Column


### More Features

### Season (Winter, April, ...)

In [69]:
manipulator.add_season('Month')


Data Manipulatior:INFO->Successfully Added Season Column


In [70]:
## Added dataframe status
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 26 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  uint16 
 1   DayOfWeek                  1017209 non-null  uint8  
 2   WeekDay                    1017209 non-null  int64  
 3   Date                       1017209 non-null  object 
 4   Year                       1017209 non-null  uint16 
 5   Month                      1017209 non-null  uint8  
 6   Season                     1017209 non-null  object 
 7   Day                        1017209 non-null  uint8  
 8   MonthTiming                1017209 non-null  int64  
 9   Sales                      1017209 non-null  uint16 
 10  Customers                  1017209 non-null  uint16 
 11  Open                       1017209 non-null  uint8  
 12  Promo                      1017209 non-null  uint8  
 13  StateHoliday

## Drop Columns

In [71]:
# Store	Date	Customers
combined_df.drop(['Store','Date','Customers'],axis=1,inplace=True)

In [72]:
## Dropped dataframe status
combined_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 23 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   DayOfWeek                  1017209 non-null  uint8  
 1   WeekDay                    1017209 non-null  int64  
 2   Year                       1017209 non-null  uint16 
 3   Month                      1017209 non-null  uint8  
 4   Season                     1017209 non-null  object 
 5   Day                        1017209 non-null  uint8  
 6   MonthTiming                1017209 non-null  int64  
 7   Sales                      1017209 non-null  uint16 
 8   Open                       1017209 non-null  uint8  
 9   Promo                      1017209 non-null  uint8  
 10  StateHoliday               1017209 non-null  object 
 11  DaysAfterHoliday           1017209 non-null  int64  
 12  DaysToHoliday              1017209 non-null  int64  
 13  SchoolHolida

# Save UnLabeled Data

In [ ]:
unlabeled_data = DataCleaner(combined_df)
unlabeled_data.save_clean_data('../data/train.csv')


## Encoding Data to Numbers

In [73]:
# Label Object type data columns (StoreType,Assortment,StateHoliday,Season)

manipulator.label_columns(['Season', 'StateHoliday', 'StoreType', 'Assortment'])
info.get_object_columns()


['PromoInterval']

In [74]:
combined_df['PromoInterval'].value_counts()

Jan,Apr,Jul,Oct     801153
Feb,May,Aug,Nov     118596
Mar,Jun,Sept,Dec     97460
Name: PromoInterval, dtype: int64

In [75]:
manipulator.label_columns(['PromoInterval'])
info.get_object_columns()


[]

In [76]:
## Label Encoded dataframe status
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 23 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   DayOfWeek                  1017209 non-null  uint8  
 1   WeekDay                    1017209 non-null  int64  
 2   Year                       1017209 non-null  uint16 
 3   Month                      1017209 non-null  uint8  
 4   Season                     1017209 non-null  int32  
 5   Day                        1017209 non-null  uint8  
 6   MonthTiming                1017209 non-null  int64  
 7   Sales                      1017209 non-null  uint16 
 8   Open                       1017209 non-null  uint8  
 9   Promo                      1017209 non-null  uint8  
 10  StateHoliday               1017209 non-null  int32  
 11  DaysAfterHoliday           1017209 non-null  int64  
 12  DaysToHoliday              1017209 non-null  int64  
 13  SchoolHolida

## Scaling Data

In [77]:
# Using StandardScaler to standardize the all columns
scale_list = combined_df.columns.to_list()
# Scale
manipulator.standardize_columns(scale_list)
info.get_min_max_of_dataframe_columns()

,Max Value,Min Value
DayOfWeek,1.502791,-1.501129
WeekDay,0.630672,-1.585611
Year,1.502077,-1.070616
Month,1.849989,-1.457193
Season,1.333709,-1.498283
Day,1.740766,-1.673123
MonthTiming,1.205894,-1.236292
Sales,9.292957,-1.499723
Open,0.452399,-2.210440
Promo,1.273237,-0.785400


# Save Clean Data

In [30]:
cleaner = DataCleaner(info.df)
cleaner.optimize_df()
cleaner.save_clean_data('../data/train.csv')